# Partie 3: Outils avancés d'exécution à distance

Dans la dernière section, nous avons formé un modèle de jouet en utilisant l'apprentissage fédéré. Nous l'avons fait en appelant .send () et .get () sur notre modèle, en l'envoyant à l'emplacement des données d'entraînement, en le mettant à jour, puis en le rapportant. Cependant, à la fin de l'exemple, nous avons réalisé que nous devions aller un peu plus loin pour protéger la vie privée des gens. A savoir, nous voulons faire la moyenne des gradients **avant** d'appeler .get (). De cette façon, nous ne verrons jamais le gradient exact de personne (protégeant ainsi mieux sa vie privée !!!)

Mais pour ce faire, nous avons besoin de quelques pièces supplémentaires:

- utiliser un pointeur pour envoyer un tenseur directement à un autre travailleur

Et en plus, pendant que nous sommes ici, nous allons en apprendre davantage sur quelques opérations de tenseur plus avancées qui nous aideront à la fois avec cet exemple et quelques-unes à l'avenir!

Auteurs:
- Andrew Trask - Twitter: [@iamtrask] (https://twitter.com/iamtrask)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# Section 3.1 - Pointeurs vers pointeurs

Comme vous le savez, les objets PointerTensor ressemblent à des tenseurs normaux. En fait, ils sont _tellement semblables à des tenseurs_ que nous pouvons même avoir des pointeurs **à** les pointeurs. Vérifiez-le!

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# c'est un tenseur local
x = torch.tensor([1,2,3,4])
x

In [ ]:
# cela envoie le tenseur local à Bob
x_ptr = x.send(bob)

# c'est maintenant un pointeur
x_ptr

In [ ]:
# maintenant nous pouvons ENVOYER LE POINTEUR à alice !!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### Qu'est-il arrivé?

Ainsi, dans l'exemple précédent, nous avons créé un tenseur appelé «x» et l'envoyer à Bob, créant un pointeur sur notre machine locale («x _ptr»).

Ensuite, nous avons appelé `x_ ptr.send (alice)` qui **a envoyé le pointeur** à Alice.

Remarque, cela n'a PAS déplacé les données! Au lieu de cela, il a déplacé le pointeur vers les données !!

In [ ]:
# Comme vous pouvez le voir ci-dessus, Bob a toujours les données réelles 
# (les données sont toujours stockées dans un type LocalTensor).
bob._objects

In [ ]:
# Alice, en revanche, a x_ptr !! (remarquez comment ça pointe sur bob)
alice._objects

In [ ]:
# et nous pouvons utiliser .get () pour récupérer x_ptr d'Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
#puis nous pouvons utiliser x_ptr pour récupérer x de Bob!

x = x_ptr.get()
x

### Arithmétique sur pointeur -> Pointeur -> Objet de données

Et tout comme avec les pointeurs normaux, nous pouvons effectuer des opérations PyTorch arbitraires sur ces tenseurs

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# Section 3.2 - Opérations de la chaîne de pointeurs

Donc, dans la dernière section, chaque fois que nous avons appelé une opération .send () ou .get (), elle a appelé cette opération directement sur le tenseur de notre machine locale. Cependant, si vous avez une chaîne de pointeurs, vous souhaitez parfois appeler des opérations telles que .get () ou .send () sur le **dernier** pointeur de la chaîne (comme l'envoi de données directement à partir d'un travailleur à un autre). Pour ce faire, vous souhaitez utiliser des fonctions spécialement conçues pour cette opération de préservation de la confidentialité.

Ces opérations sont:

- `mon _pointer2pointer.move (un autre_ travailleur)`

In [ ]:
# x est maintenant un pointeur vers les données qui vivent sur la machine de Bob
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x

Excellent! Nous sommes maintenant équipés des outils pour effectuer une **moyenne de gradient à distance** à l'aide d'un agrégateur de confiance!

# Toutes nos félicitations!!! - Il est temps de rejoindre la communauté!

Félicitations pour avoir terminé ce didacticiel pour ordinateur portable! Si cela vous a plu et que vous souhaitez rejoindre le mouvement vers la préservation de la vie privée, la propriété décentralisée de l'IA et la chaîne d'approvisionnement de l'IA (données), vous pouvez le faire de la manière suivante!

### Star PySyft sur GitHub

La façon la plus simple d'aider notre communauté est de mettre en vedette les Repos! Cela permet de mieux faire connaître les outils sympas que nous construisons.

- [Star PySyft] (https://github.com/OpenMined/PySyft)

### Rejoignez-nous sur Slack!

La meilleure façon de vous tenir au courant des dernières avancées est de rejoindre notre communauté! Vous pouvez le faire en remplissant le formulaire à [http://slack.openmined.org] (http://slack.openmined.org)

### Rejoignez-nous sur un projet de code!

La meilleure façon de contribuer à notre communauté est de devenir un contributeur de code! À tout moment, vous pouvez accéder à la page Problèmes de PySyft GitHub et filtrer pour "Projets". Cela vous montrera tous les billets de haut niveau donnant un aperçu des projets que vous pouvez rejoindre! Si vous ne souhaitez pas rejoindre un projet, mais que vous souhaitez faire un peu de codage, vous pouvez également rechercher d'autres mini-projets "uniques" en recherchant les problèmes GitHub marqués "bon premier problème".

- [Projets PySyft] (https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Billets Good First Issue] (https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Faire un don

Si vous n'avez pas le temps de contribuer à notre base de code, mais souhaitez tout de même apporter votre soutien, vous pouvez également devenir Backer sur notre Open Collective. Tous les dons vont à notre hébergement Web et à d'autres dépenses communautaires telles que les hackathons et les rencontres!

[Open Collective Page d'OpenMined] (https://opencollective.com/openmined)